# Library

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

# [로드] 등록&제원 정보 2022.12.28 기준 

In [2]:
ods_fold = 'D:/data/big2/ODS'

In [3]:
# about 8m 42s
file_name = '[ODS]등록정보&제원정보(2022.12.29).csv'
df = pd.read_csv(os.path.join(ods_fold, file_name), encoding='cp949', low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29534047 entries, 0 to 29534046
Data columns (total 49 columns):
 #   Column        Dtype  
---  ------        -----  
 0   차량관리번호        object 
 1   차량등록번호        object 
 2   본거지법정동코드      int64  
 3   소유자구분         object 
 4   차종_등록정보       object 
 5   용도            object 
 6   현재회원번호        object 
 7   차대번호          object 
 8   최초등록일자        int64  
 9   차량연식          int64  
 10  차량제작일자        float64
 11  배출가스검사유효일자    float64
 12  제원관리번호        object 
 13  배출가스인증번호      object 
 14  배출가스등급        object 
 15  저공해조치YN(미사용)  object 
 16  저공해조치종류       float64
 17  저공해조치구조변경YN   object 
 18  차량등록번호숫자정보    int64  
 19  차량말소YN        object 
 20  최초회원번호        object 
 21  취득일자          object 
 22  취득금액          float64
 23  생성일시_등록정보     object 
 24  수정일시_등록정보     object 
 25  차명            object 
 26  차종_제원정보       object 
 27  차종분류          object 
 28  차종유형(소분류)     object 
 29  자동차형식         object 
 30  제작사명          ob

In [4]:
df['본거지법정동코드'].head()

0    1165010700
1    2711010500
2    2820010300
3    4213011200
4    4812110500
Name: 본거지법정동코드, dtype: int64

# [로드] 본거지법정동코드

In [5]:
fold = 'D:/data/big2'

In [6]:
file_name = '법정동코드_2022.10.14.xlsx'
code = pd.read_excel(os.path.join(fold, file_name))
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46340 entries, 0 to 46339
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   법정동코드     46340 non-null  int64  
 1   시도        46340 non-null  object 
 2   시구군       46317 non-null  object 
 3   읍면동       45847 non-null  object 
 4   리         33682 non-null  object 
 5   지역3       0 non-null      float64
 6   폐지여부      46340 non-null  object 
 7   대기관리권역    46340 non-null  object 
 8   대기관리권역YN  46334 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 3.2+ MB


In [7]:
code['법정동코드'].head()

0    1100000000
1    1111000000
2    1111010100
3    1111010200
4    1111010300
Name: 법정동코드, dtype: int64

# [merge] 법정동코드 기준

In [8]:
# '본거지법정도코드'열 데이터 확인
df['본거지법정동코드'].isnull().sum()

0

In [9]:
df.shape

(29534047, 49)

In [11]:
# about 39s
tot = df.merge(code, left_on='본거지법정동코드', right_on='법정동코드', how='left')
tot.shape

(29534047, 58)

In [12]:
len(tot['차대번호'].unique())

29521505

In [13]:
tot['차대번호'].isnull().sum()

0

# 전처리

## 말소제거
- 차량말소YN

In [16]:
# about 
tot1 = tot[tot['차량말소YN'] == 'N'].reset_index(drop=True)
tot1.shape

(25693532, 58)

In [59]:
tot1.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보', '법정동코드',
       '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

In [60]:
tot1_1 = tot1.sort_values(['최초등록일자', '배출가스검사유효일자'], ascending=[False, False]).drop_duplicates(['차대번호']).reset_index(drop=True)
tot1_1.shape

(25692512, 58)

In [ ]:
tot1_1['배출가스등급'].value_counts(dropna=False)

## 폐지여부 확인

In [17]:
tot1['폐지여부'].value_counts(dropna=False)

존재     25611213
폐지        82317
NaN           2
Name: 폐지여부, dtype: int64

In [18]:
tot2 = tot1[tot1['폐지여부'] == '존재'].reset_index(drop=True)
tot2.shape

(25611213, 58)

## 법정동코드 확인

In [19]:
tot2['법정동코드'].isnull().sum()

0

## 연료 확인

In [ ]:
tot2['연료'].isnull().sum()

146468

In [43]:
tot2['연료'].value_counts(dropna=False)

휘발유            12138821
경유              9855813
LPG(액화석유가스)     1899359
휘발유 하이브리드       1092740
전기               368150
NaN              146468
경유 하이브리드          35727
CNG(압축천연가스)       30372
수소                28199
LPG 하이브리드         14301
기타연료                926
CNG 하이브리드           304
LNG(액화천연가스)          27
알코올                   3
태양열                   2
등유                    1
Name: 연료, dtype: int64

In [45]:
tot2.loc[tot2['연료'] == '기타연료', '엔진형식'].unique()

array([nan, '3304124', '08784250915', '09801220246', '10784203005',
       '10784203004', 'LNS', '0', '01821300581', 'G4AJ', 'G15CF', 'B5',
       'H20MR', 'G16SF', 'MSV3', 'ZB', '-', '1NZ', '2AZ', 'RW-300',
       'M1082', '.'], dtype=object)

In [ ]:
# '3304124' : -
# '08784250915' : -
# '09801220246' : -
# '10784203005' : -
# '10784203004' : -
# 'LNS' : -
# '0' : -
# '01821300581' : -
# 'G4AJ' : 휘발유
# 'G15CF' : 휘발유
# 'B5' : 휘발유
# 'H20MR' : 휘발유
# 'G16SF' : 휘발유
# 'MSV3' : 휘발유
# 'ZB' : 디젤
# '-' : -
# '1NZ' : 휘발유
# '2AZ' : 휘발유
# 'RW-300' : -
# 'M1082' : -
# '.' : -

In [47]:
engine_form = ['G4AJ', 'G15CF', 'B5', 'H20MR', 'G16SF', 'MSV3', 'ZB', '1NZ', '2AZ']
len(engine_form)

9

In [48]:
for one in engine_form:
    print(tot2[tot2['엔진형식'] == one].shape)

(5031, 58)
(3235, 58)
(16029, 58)
(2896, 58)
(46, 58)
(1091, 58)
(5834, 58)
(2352, 58)
(1358, 58)


In [ ]:
tot2.loc[tot2['엔진형식'] == 'G4AJ', '연료'] = '휘발유'
tot2.loc[tot2['엔진형식'] == 'G15CF', '연료'] = '휘발유'
tot2.loc[tot2['엔진형식'] == 'B5', '연료'] = '휘발유'
tot2.loc[tot2['엔진형식'] == 'H20MR', '연료'] = '휘발유'
tot2.loc[tot2['엔진형식'] == 'G16SF', '연료'] = '휘발유'
tot2.loc[tot2['엔진형식'] == 'MSV3', '연료'] = '휘발유'
tot2.loc[tot2['엔진형식'] == 'ZB', '연료'] = '경유'
tot2.loc[tot2['엔진형식'] == '1NZ', '연료'] = '경유'
tot2.loc[tot2['엔진형식'] == '2AZ', '연료'] = '경유'

In [50]:
car_name = ['도요다크라운로얄싸롱', '스텔라오토매틱', '벤즈 230E', '벤즈230E']
len(car_name)

4

In [51]:
for one in car_name:
    print(tot2[tot2['차명'] == one].shape)

(20, 58)
(76, 58)
(1, 58)
(5, 58)


In [53]:
tot2['연료'].value_counts(dropna=False)

휘발유            12138821
경유              9855813
LPG(액화석유가스)     1899359
휘발유 하이브리드       1092740
전기               368150
NaN              146468
경유 하이브리드          35727
CNG(압축천연가스)       30372
수소                28199
LPG 하이브리드         14301
기타연료                926
CNG 하이브리드           304
LNG(액화천연가스)          27
알코올                   3
태양열                   2
등유                    1
Name: 연료, dtype: int64

In [52]:
tot2.loc[tot2['연료'] == '기타연료', '차명'].unique()

array(['미스비시샤시(세미트레일러)', '다이아몬드(추레라)피견인차', '트레라', '세미트레라', '트레일러',
       '세미트레일러', '쎄미트레라', '일산 육륜', '히노 트레라', '피견인차(탱크로리)', '히노탱크로리트레라',
       '히노탱크로리', '코티나마크IV', '후소트레일러', '가와사끼샤시(세미트레일러)', '도큐트레일러',
       '세미 트레일러', '콘테이너샤시콤바인', '콘테이너샤시트레일러', '인터내쇼날트레일러', '추레라',
       '프렛트홈트레일러콤바인', '프렛트홈트레라콤바인', '제너럴트레일러', '아시아AM707중형버스',
       '훌러하프탱크추레라', '후루하프트레라', '드라어  추레라', '현대708콘테이너전용트레일러', '인타담프',
       '동아저상식트레일러', '저상식트레일러', '브리샤2', '추레라(피견인차)', '도큐사요트레러', '새한버스',
       '다이하스웨곤', '스트리크트레일러샤시', '프랫트홈트레일러콤바인', '세미25톤트레일러', '탱크로리트레라',
       '도요다웨곤엠브란스', '도요다크라운', '추례라', '진도콤비네이션콘테너샤시', '쎄미트레일러', '로얄살롱',
       '도요다코로나', '스텔라', '스텔라1.6CXL', '골드호퍼트레일러', '대우11톤카고',
       '현대401드롭배드트레일러', '대우8톤카고', '맥스디젤삼방', '현대리어엔진버스', '대우26톤드롭베스트레일러',
       '포니2', '동아덤프트레일러', '도요다크라운로얄싸롱', '현대구즈넥콘테이너트레일러', '현대평판트레일러',
       '현대20KL스텐탱크트레일러', '23톤구즈넥덤프트레일러', '구즈넥콘테이너샤시', '26톤드톱베드트레일러',
       '구급차(도요다)', '현대콘테이너트레일러', '동아콘테이너샤시', '진도구즈넥콘테이너샤시', '진도프레트폼트레일러',
       '진도20피드콘테이저샤시', '25M3벌크시멘트트레일러', 

In [56]:
[
    '미스비시샤시(세미트레일러)', #
    '다이아몬드(추레라)피견인차', #
    '트레라', #
    '세미트레라', #
    '트레일러', #
    '세미트레일러', #
    '쎄미트레라', #
    '일산 육륜', 
    '히노 트레라', #
    '피견인차(탱크로리)', #
    '히노탱크로리트레라', #
    '히노탱크로리', # 경유
    '코티나마크IV', # 휘발유
    '후소트레일러', #
    '가와사끼샤시(세미트레일러)', #
    '도큐트레일러', #
    '세미 트레일러', #
    '콘테이너샤시콤바인', #
    '콘테이너샤시트레일러', #
    '인터내쇼날트레일러', #
    '추레라', #
    '프렛트홈트레일러콤바인', #
    '프렛트홈트레라콤바인', #
    '제너럴트레일러', #
    '아시아AM707중형버스',
    '훌러하프탱크추레라',  #
    '후루하프트레라', #
    '드라어  추레라', #
    '현대708콘테이너전용트레일러', #
    '인타담프', # 경유
    '동아저상식트레일러', #
    '저상식트레일러', #
    '브리샤2', # 휘발유
    '추레라(피견인차)', #
    '도큐사요트레러', #
    '새한버스', # 경유
    '다이하스웨곤', # 휘발유
    '스트리크트레일러샤시', #
    '프랫트홈트레일러콤바인', #
    '세미25톤트레일러', #
    '탱크로리트레라', #
    '도요다웨곤엠브란스', 
    '도요다크라운', # 휘발유
    '추례라', #
    '진도콤비네이션콘테너샤시', #
    '쎄미트레일러', #
    '로얄살롱', # 휘발유
    '도요다코로나', # 휘발유
    '스텔라', # 휘발유
    '스텔라1.6CXL', 
    '골드호퍼트레일러', 
    '대우11톤카고',
    '현대401드롭배드트레일러', 
    '대우8톤카고', 
    '맥스디젤삼방', 
    '현대리어엔진버스', 
    '대우26톤드롭베스트레일러',
    '포니2', 
    '동아덤프트레일러', 
    '도요다크라운로얄싸롱', 
    '현대구즈넥콘테이너트레일러', 
    '현대평판트레일러',
    '현대20KL스텐탱크트레일러', 
    '23톤구즈넥덤프트레일러', 
    '구즈넥콘테이너샤시', 
    '26톤드톱베드트레일러',
    '구급차(도요다)', 
    '현대콘테이너트레일러', 
    '동아콘테이너샤시', 
    '진도구즈넥콘테이너샤시', 
    '진도프레트폼트레일러',
    '진도20피드콘테이저샤시', 
    '25M3벌크시멘트트레일러', 
    "현대45'익스텐션콘테이너트", 
    '벤즈230E',
    '진도구즈넥슬라이더콤비샤시', 
    '프레스토오토매틱', 
    '르망펜타-5', 
    '진도콤비네이션콘테이너샤시', 
    '스텔라오토매틱',
    '엑셀', 
    '벤즈 230E', 
    '진도구즈넥슬라이더콘테이너', 
    '동아콘테이너샤시콤바인', 
    '캐피탈',
    '로얄살롱2.0AT', 
    '르망', 
    '현대45`익스텐션콘테이너트', 
    '현대45,익스텐션콘테이너트',
    '25M3벌크트레일러', 
    '23M3 벌크트레일러', 
    '머큐리 세이블', 
    '트레일러  프렛홈형',
    '디엠티 탱크 트레일러', 
    '복사14', 
    'HYUNDAI PRESTO', 
    '미래40피트콤비네이션콘테이너샤시',
    '도요타 프리우스(하이브리드)', 
    'TOYOTA CAMRY하이브리드', 
    '보트트레일러', 
    '장보고 보트 트레일러',
    '동우 가변형 평판트레일러', 
    'XW300', 
    '대한평판트레일러', 
    '대한콤비컨테이너트레일러',
    '바인스베르크400LK캠핑트레일러', 
    '미래40피트틸팅컨테이너샤시', 
    '소노마캠팡트레일러',
    '에어스트림탐지견수송트레일러', 
    '이보1850캠핑트레일러', 
    'RW300', 
    'WSR레저장비운송트레일러',
    '트레일마스터보트트레일러', 
    '제니스5캠핑트레일러', 
    'ALTEA542캠핑트레일러', 
    '트레일러-LMTV',
    '카고 트레일러',
    'TRAILER CARGO LMTV', 
    'D.K트레일러', 
    'FEISHEN일반화물운송트레일러',
    'SID 500 F1', 
    '카라반KUS2960평판트레일러', 
    '밤챔버용 트레일러', 
    '아우토니드이동가축관리트레일러',
    '렉스온캠핑트레일러', 
    '포레스트리버캠핑트레일러', 
    '홍성푸드트레일러', 
    '펜트(FENDT)캠핑트레일러',
    '밀폐형 방폭트레일러(DynaSEALR X10 LTR on Trailer)', 
    '홍성캠핑트레일러',
    '아드리아(ADRIA)캠핑트레일러', 
    '트리가노(TRIGANO)스탈렛캠핑트레일러', 
    '엘디스(ELDDIS)캠핑트레일러',
    '베일리(BAILEY)캠핑트레일러', 
    '바인스버그(WEINSBERG) 캠핑트레일러', 
    '미래40피트평판트레일러',
    '타버트(TABBERT)캠핑트레일러', 
    '씨티인터내셔널캠핑트레일러'
]

['미스비시샤시(세미트레일러)',
 '다이아몬드(추레라)피견인차',
 '트레라',
 '세미트레라',
 '트레일러',
 '세미트레일러',
 '쎄미트레라',
 '일산 육륜',
 '히노 트레라',
 '피견인차(탱크로리)',
 '히노탱크로리트레라',
 '히노탱크로리',
 '코티나마크IV',
 '후소트레일러',
 '가와사끼샤시(세미트레일러)',
 '도큐트레일러',
 '세미 트레일러',
 '콘테이너샤시콤바인',
 '콘테이너샤시트레일러',
 '인터내쇼날트레일러',
 '추레라',
 '프렛트홈트레일러콤바인',
 '프렛트홈트레라콤바인',
 '제너럴트레일러',
 '아시아AM707중형버스',
 '훌러하프탱크추레라',
 '후루하프트레라',
 '드라어  추레라',
 '현대708콘테이너전용트레일러',
 '인타담프',
 '동아저상식트레일러',
 '저상식트레일러',
 '브리샤2',
 '추레라(피견인차)',
 '도큐사요트레러',
 '새한버스',
 '다이하스웨곤',
 '스트리크트레일러샤시',
 '프랫트홈트레일러콤바인',
 '세미25톤트레일러',
 '탱크로리트레라',
 '도요다웨곤엠브란스',
 '도요다크라운',
 '추례라',
 '진도콤비네이션콘테너샤시',
 '쎄미트레일러',
 '로얄살롱',
 '도요다코로나',
 '스텔라',
 '스텔라1.6CXL',
 '골드호퍼트레일러',
 '대우11톤카고',
 '현대401드롭배드트레일러',
 '대우8톤카고',
 '맥스디젤삼방',
 '현대리어엔진버스',
 '대우26톤드롭베스트레일러',
 '포니2',
 '동아덤프트레일러',
 '도요다크라운로얄싸롱',
 '현대구즈넥콘테이너트레일러',
 '현대평판트레일러',
 '현대20KL스텐탱크트레일러',
 '23톤구즈넥덤프트레일러',
 '구즈넥콘테이너샤시',
 '26톤드톱베드트레일러',
 '구급차(도요다)',
 '현대콘테이너트레일러',
 '동아콘테이너샤시',
 '진도구즈넥콘테이너샤시',
 '진도프레트폼트레일러',
 '진도20피드콘테이저샤시',
 '25M3벌크시멘트트레일러',
 "현대45'익스텐션콘테이너트",
 '벤즈230E',
 '

In [ ]:
tot2.loc[tot2['차명'] == '도요다크라운로얄싸롱', '연료'] = '휘발유'
tot2.loc[tot2['차명'] == '스텔라오토매틱', '연료'] = '휘발유'
tot2.loc[tot2['차명'] == '벤즈 230E', '연료'] = '휘발유'
tot2.loc[tot2['차명'] == '벤즈230E', '연료'] = '휘발유'

## 2022년 6월 기준으로 X등급 샘플

# 분석

##  연료별 통계

In [20]:
tot2.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보', '법정동코드',
       '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

In [21]:
tot2.groupby(['연료', '배출가스등급'], dropna=False)['차대번호'].count()

연료           배출가스등급
CNG 하이브리드    1            287
             2             17
CNG(압축천연가스)  2          23282
             3           5989
             4              1
                        ...  
NaN          2             26
             3            138
             4             71
             5             29
             X         146204
Name: 차대번호, Length: 65, dtype: int64

In [34]:
tot2['배출가스등급'].isnull().sum()

0

In [57]:
tot2[tot2['배출가스등급'] == 'X'].shape

(288615, 58)

In [22]:
tot2.groupby(['연료', '배출가스등급'], dropna=False)['차대번호'].count().unstack()

배출가스등급,1,2,3,4,5,T2,T3,T4,T5,X
연료,,,,,,,,,,
CNG 하이브리드,287.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CNG(압축천연가스),NaN,23282.0,5989.0,1.0,5.0,945.0,NaN,NaN,NaN,150.0
LNG(액화천연가스),NaN,2.0,NaN,NaN,NaN,23.0,NaN,NaN,NaN,2.0
LPG 하이브리드,14298.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0
LPG(액화석유가스),460969.0,1161049.0,235912.0,26018.0,433.0,7395.0,NaN,NaN,NaN,7583.0
경유,NaN,NaN,7437223.0,1148915.0,1119808.0,NaN,119825.0,29.0,18.0,29995.0
경유 하이브리드,NaN,NaN,18039.0,NaN,NaN,NaN,16100.0,NaN,NaN,1588.0
기타연료,NaN,NaN,3.0,36.0,20.0,NaN,NaN,3.0,5.0,859.0
등유,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [39]:
tot2[tot2['배출가스등급'] == '2'].shape

(10925973, 58)

In [35]:
tot2[tot2['배출가스등급'] == 'T2'].shape

(246992, 58)

In [40]:
tot2[tot2['배출가스등급'] == '3'].shape

(8592960, 58)

In [36]:
tot2[tot2['배출가스등급'] == 'T3'].shape

(135973, 58)

In [41]:
tot2[tot2['배출가스등급'] == '4'].shape

(1505895, 58)

In [37]:
tot2[tot2['배출가스등급'] == 'T4'].shape

(37, 58)

In [42]:
tot2[tot2['배출가스등급'] == '5'].shape

(1150131, 58)

In [38]:
tot2[tot2['배출가스등급'] == 'T5'].shape

(28, 58)

In [23]:
tot2['시도'].unique()

array(['서울', '대구', '인천', '강원', '경남', '전북', '충북', '경기', '부산', '대전', '울산',
       '경북', '제주', '전남', '광주', '충남', '세종'], dtype=object)

In [24]:
tot2['시도'].isnull().sum()

0

In [25]:
tot2['대기관리권역'].unique()

array(['수도권', '동남권', '그 외', '중부권', '남부권'], dtype=object)

In [26]:
tot2['대기관리권역'].isnull().sum()

0

In [27]:
tot2['대기관리권역YN'].unique()

array(['Y', 'N', nan], dtype=object)

In [28]:
tot2['대기관리권역YN'].isnull().sum()

1

In [31]:
tot2.loc[tot2['대기관리권역YN'].isnull() == True, ['대기관리권역', '시도', '시구군', '읍면동', '리', '지역3']]

,대기관리권역,시도,시구군,읍면동,리,지역3
6139,수도권,경기,NaN,NaN,NaN,NaN
